## 4. Preparing to use image services...


In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
import requests, io, astropy
from IPython.display import Image, display

## For handling ordinary astropy Tables
from astropy.table import Table, vstack

## For reading FITS files
import astropy.io.fits as apfits

## There are a number of relatively unimportant warnings that 
## show up, so for now, suppress them:
import warnings
warnings.filterwarnings("ignore")

## our stuff
import sys
# Use the NASA_NAVO/astroquery
from navo_utils.cone import Cone
from navo_utils.registry import Registry
from navo_utils.registry import display_results    ### Make better -- create table with #, URL, etc. so we can grab output XX as the access_url.
import navo_utils.image as image
#from navo_utils.image import display_info
import astroquery

## First we need to know what registry services have the image data that we want. The Registry query function is useful here for picking out the "image" (SIA) service. Here, we define a shortcut to query the registry and output a list of the MAST data archives that contain ultraviolet wavelength data. However, you can easily select optical infrared, or any other wavelengths by modifying the "waveband" parameter or select irsa, heasarc, or leave the "source" field blank if you want other data archives. 

## This step is like deciding which website to go to to access your image data. But this method will give you greater flexibility. If you know the access_url for your data, you can skip this step. 

In [2]:
def list_image_services(**kwargs):
    return Registry.query(service_type="image",**kwargs)
results=list_image_services(waveband="uv", source="mast") # THIS WORKS

print('Found {} results:'.format(len(results)))

print(results.columns)
ivoid=results[:]['ivoid']
url=results[:]['access_url']

# For visual reference:
for i in range(len(url)):
    print(i, ':', ivoid[i], '; ', url[i])


Found 12 results:
<TableColumns names=('waveband','short_name','ivoid','res_description','access_url','reference_url')>
0 : ivo://mast.stsci/acsggct ;  http://archive.stsci.edu/siap/search.php?id=acsggct&amp;
1 : ivo://mast.stsci/hippies ;  http://archive.stsci.edu/siap/search.php?id=HIPPIES&amp;
2 : ivo://mast.stsci/siap/uit ;  http://archive.stsci.edu/siap/search.php?id=UIT&amp;
3 : ivo://mast.stsci/stpr ;  http://archive.stsci.edu/stpr/vo_search.php?
4 : ivo://mast.stsci/siap/galex_atlas ;  http://archive.stsci.edu/siap/search.php?id=galex_atlas&amp;
5 : ivo://mast.stsci/candels ;  http://archive.stsci.edu/siap/search.php?id=candels&amp;
6 : ivo://mast.stsci/phat ;  http://archive.stsci.edu/siap/search.php?id=phat&amp;
7 : ivo://mast.stsci/siap/hst.maoz_atlas ;  http://archive.stsci.edu/siap/search.php?id=maoz_atlas&amp;
8 : ivo://mast.stsci/clash ;  http://archive.stsci.edu/siap/search.php?id=clash&amp;
9 : ivo://mast.stsci/angrrr ;  http://archive.stsci.edu/siap/search.php?id=angr

## 5 SIA

We find the relevant image data within catalogs containing image data. 

### 5.1 Using a specific data catalog to search for image data for a source
Once you find which data archive contains data that you are interested in, you can find available image data from an SIA service using the access_url. 

For example, say that we know the CANDELS data is best for our purpose. This corresponds to the number 5. Here we show how you get a table with information about images within this data catalog for sources within 5 arcsec of the given ra,dec position. 

In [4]:
### TESTING.... comment out section for original MAST example
results=list_image_services(waveband="x-ray", source="heasarc") # THIS breaks things... 

# print(results.columns)
# ivoid=results[:]['ivoid']
# url=results[:]['access_url']

# For visual reference:
# for i in range(len(url)):
#    print(i, ':', ivoid[i], '; ', url[i])
    
#im_info=sia_query(access_url=results[31]['access_url'], ra='189.22917', dec='62.23830', size='.001') ### FAILS!
#results=list_image_services(waveband="infrared", source="irsa") # THIS breaks things... 

#print(results.columns)
#ivoid=results[:]['ivoid']
#url=results[:]['access_url']

# For visual reference:
#for i in range(len(url)):
#    print(i, ':', ivoid[i], '; ', url[i])
#im_info=sia_query(access_url=results[20]['access_url'], ra='189.22917', dec='62.23830', size='.001') ### FAILS! (File Does not appear to be VOTable)
access_url=results[20]['access_url']
from navo_utils.utils import sval, astropy_table_from_votable_response
ra='189.22917'
dec='62.23830'
size='0.001'
image_formats='ALL'
params={
        'POS': sval(ra) + ',' + sval(dec),
        'SIZE': sval(size),
        'FORMAT': image_formats
    }
results = requests.get(access_url, params)
content = results.content
print(content)
#for table in results:
#    str1=repr(table)
#    # print(str1)
#    print('No images' in str1)
#    if ('No images' in str1 == True):
#        print('OOPS!')
#test=astropy_table_from_votable_response(results)
#print(test.columns)
############### UNCOMMENT FOR ORIGINAL EXAMPLE
im_info=sia_query(access_url=results[5]['access_url'], ra='189.22917', dec='62.23830', size='.001')

# For visual reference:
#print(im_info.columns)
#print()
#print('Found {} results:'.format(len(im_info)))
#print()
#for i in range(len(im_info)):
#    print(im_info[i]['ra_j2000'], im_info[i]['dec_j2000'],  im_info[i]['bandpass_id'], im_info[i]['url'])

b'<?xml version="1.0"?>\n<VOTABLE xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.ivoa.net/xml/VOTable/v1.0" xsi:schemaLocation="http://www.ivoa.net/xml/VOTable/v1.0 http://www.ivoa.net/xml/VOTable/VOTable-1.0.xsd" version="1.0">\n <RESOURCE type="results">\n  <INFO name="Info:Name resolution" value="Position 189.22917,62.23830 corresponds to J2000 decimal coordinates:189.22916999999998 62.2383"/>\n  <INFO name="QUERY_STATUS" value="OK"/>\n  <TABLE>\n   <DESCRIPTION>Query result for:&#xA;select detection_name, ra, dec, count_rate, count_rate_error, exposure, offaxis, srcflags, seqflags, __row,  \'/FTP/rosat/data/hri/processed_data/\'   || substring(  seqid  , 3, 1) ||   \'00000/\'   || lower(  seqid  ) ||   \'/\'  as "__p_roshritotal_rosat_obs_hri_root",  \'HRI\'  as "__s_roshritotal_rosat_obs_hri_im_im1_gif_Instrument",  \'HRI\'  as "__s_roshritotal_rosat_obs_hri_im_im2p_gif_Instrument",  \'HRI\'  as "__s_roshritotal_rosat_obs_hri_im_im2p_fits_Instrument",  \'H

NameError: name 'sia_query' is not defined

### 5.2 Searching for image data for a given source in all available catalogs
Perhaps you don't know which service is quite what you want, so you get a table list for all images in a list of services that contain a given source. Remember that this is already selected for MAST and UV images based on the registry query function above. 


In [133]:
#print(results[:]['access_url'])
allim_info=sia_query(access_url=results[11]['access_url'], ra='189.22917', dec='62.23830', size='.001')
############# OK, Dunno Why... but this is breaking with IRSA, HEASARC... and maybe others? Works with MAST.

# For visual reference:
print(allim_info.columns)
print()
print('Found {} results:'.format(len(allim_info)))
print()
for i in range(len(allim_info)):
    print(allim_info[i]['ra_j2000'], allim_info[i]['dec_j2000'],  allim_info[i]['bandpass_id'], allim_info[i]['url'])

<TableColumns names=('name','ra','dec','time','obsid','exposure','processing_date','public_date','SIA_title','SIA_scale','SIA_naxis','SIA_naxes','SIA_format','SIA_reference','SIA_ra','SIA_dec','SIA_instrument')>

Found 0 results:



### 5.3 OPTION 2: SEARCH USING POSITION FOR SERVICES THAT CONTAIN SOURCE OF INTEREST...
Or, like with the Cone search above, the user doesn't give a service but just asks for information on images matching whatever criteria:

    heasarc_images_info = astroquery.vo.Image.query( 
        pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’, service='heasarc')
    uv_images_info = astroquery.vo.Image.query( 
        pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’, waveband='uv')
    2mass_images_info = astroquery.vo.Image.query( 
        pos=‘185.47873,4.47365', size=‘0’, naxis=‘300,300’, keyword='2mass')


## 7-8. Retrieving images

You can look at the images_info and pick one to download:

    astroquery.vo.Image.get( images_info[6], filename='my_file.fits')
    
or get the image data to hand to the plotter:

    image=astroquery.vo.Image.get( images_info[6] )
    plt.imshow( image,  cmap='gray', origin='lower',vmax=0.02 )
    
where it downloads it to filename if specified, or to a temporary filename if not and then reads it in and returns the image data if not.

Or hand a list and return a list of images:

    images=astroquery.vo.Image.get(images_info )
    plt.imshow( images[0],  cmap='gray', origin='lower',vmax=0.02 )

In [11]:
class ImageClass(BaseQuery):
    
    def query(self, **kwargs):
        """Get information on what images are available"""
        services=Registry.query(service_type='image',**kwargs)
        # Like the Cone class above, collect results...
        return 
    
    def get(self, image_url , filename=''):
        """Returns the data that can be handed to plt.imshow() from a URL
    
        For now, input URL. But could just get a list of URLs or a 
        list of tables that have an 'access_url' column.
        """
        if filename is '':
            filename='tmp.fits'
        self._download( image_url, filename=filename)
        if filename == 'tmp.fits':
            hdus=astropy.io.fits.open('tmp.fits')
            # Which extension? TBD
            return hdus[0].data
        else:
            return

    def _download(self, url, filename=''):
        # simple wrapper of urllib
        return
    
class SpectralClass(ImageClass):

SyntaxError: unexpected EOF while parsing (<ipython-input-11-eb5b908706e8>, line 29)